In [2]:
import torch
from transformers import AutoModel, AutoTokenizer
from pyvi import ViTokenizer
import json
from qdrant_client import QdrantClient, models

# Tải tokenizer và model SimCSE
tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


Some weights of the model checkpoint at VoVanPhuc/sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.bias', 'mlp.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]  # Lấy embedding của token [CLS]
    return embeddings.squeeze().cpu().numpy()


In [4]:
client = QdrantClient(url="http://localhost:6333")


In [5]:
with open('content_test_newquery_filter_media.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


In [6]:
text =  """Người dân địa phương đã cứu hộ thành công một bé gái 5 tuổi bị lạc trong rừng suốt hai ngày. Lực lượng cứu hộ đã phối hợp với các cơ quan chức năng và người dân quanh vùng để tìm kiếm và đưa bé về nhà an toàn."""


In [7]:
ViTokenizer.tokenize(text)

'Người dân địa_phương đã cứu_hộ thành_công một bé gái 5 tuổi bị lạc trong rừng suốt hai ngày . Lực_lượng cứu_hộ đã phối_hợp với các cơ_quan_chức_năng và người dân quanh vùng để tìm_kiếm và đưa bé về nhà an_toàn .'

In [12]:
len(encode_text(ViTokenizer.tokenize(text)).tolist())

768

In [ ]:
encode_text(ViTokenizer.tokenize(item['content'])).tolist(),  # Vector hóa 'content'


In [2]:
client.create_collection(
    collection_name="content_vectors",
    vectors_config=models.VectorParams(
        size=768,  
        distance=models.Distance.COSINE 
    ),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(always_ram=True),  
    ),
    hnsw_config=models.HnswConfigDiff(
        m=32,  
        ef_construct=100,  
        full_scan_threshold=10000  
    ),
    optimizers_config=models.OptimizersConfigDiff(
        default_segment_number=5,  # Số segment mặc định
        indexing_threshold=0,  # Lập chỉ mục ngay lập tức cho mỗi vector mới
    ),
    on_disk_payload=False  
)

True

In [3]:
client.create_collection(
    collection_name="title_vectors",
    vectors_config=models.VectorParams(
        size=768,  
        distance=models.Distance.COSINE  
    ),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(always_ram=True),  
    ),
    hnsw_config=models.HnswConfigDiff(
        m=32,  
        ef_construct=100,  
        full_scan_threshold=10000  
    ),
    optimizers_config=models.OptimizersConfigDiff(
        default_segment_number=5, 
        indexing_threshold=0,  
    ),
    on_disk_payload=False 
)


True

In [14]:
import uuid

for item in data:
    content = item['_source']['content']
    title = item['_source']['title']
    
    # Mã hóa content và title thành vector
    content_vector = encode_text(ViTokenizer.tokenize(content))
    title_vector = encode_text(ViTokenizer.tokenize(title))
    
    # Tạo UUID cho content và title
    content_id = str(uuid.uuid4())  # Tạo UUID cho content
    title_id = str(uuid.uuid4())  # Tạo UUID cho title
    
    # Thêm content vào collection 'content_vectors'
    client.upsert(
        collection_name="content_vectors",
        points=[
            models.PointStruct(
                id=content_id,  # Sử dụng UUID hợp lệ
                vector=content_vector.tolist(),  # Vector hóa 'content'
                payload={"article_id": item["_id"]}  # Lưu ID bài viết trong payload
            )
        ]
    )
    
    # Thêm title vào collection 'title_vectors'
    client.upsert(
        collection_name="title_vectors",
        points=[
            models.PointStruct(
                id=title_id,  # Sử dụng UUID hợp lệ cho title
                vector=title_vector.tolist(),  # Vector hóa 'title'
                payload={"article_id": item["_id"]}  # Lưu ID bài viết trong payload
            )
        ]
    )


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "c:\Users\Admin\anaconda3\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "c:\Users\Admin\anaconda3\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import uuid
from concurrent.futures import ThreadPoolExecutor, as_completed

# Hàm để thực hiện batch upsert
def batch_upsert(content_batch, title_batch):
    content_points = [
        models.PointStruct(
            id=str(uuid.uuid4()),  # Tạo UUID cho content
            vector=encode_text(ViTokenizer.tokenize(item['content'])).tolist(),  # Vector hóa 'content'
            payload={"article_id": item["_id"]}  # Lưu ID bài viết trong payload
        )
        for i, item in enumerate(content_batch)
    ]
    
    title_points = [
        models.PointStruct(
            id=str(uuid.uuid4()),  # Tạo UUID cho title
            vector=encode_text(ViTokenizer.tokenize(item['title'])).tolist(),  # Vector hóa 'title'
            payload={"article_id": item["_id"]}  # Lưu ID bài viết trong payload
        )
        for i, item in enumerate(title_batch)
    ]
    
    # Thực hiện upsert cho cả content và title
    client.upsert(collection_name="content_vectors", points=content_points)
    client.upsert(collection_name="title_vectors", points=title_points)

# Chia dữ liệu thành các batch
batch_size = 100  # Chọn kích thước batch hợp lý
batches = [data[i:i + batch_size] for i in range(0, len(data), batch_size)]

# Dùng ThreadPoolExecutor để chạy song song các batch
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    
    for batch in batches:
        content_batch = [{"_id": item["_id"], "content": item["_source"]["content"]} for item in batch]
        title_batch = [{"_id": item["_id"], "title": item["_source"]["title"]} for item in batch]
        
        # Thực hiện batch upsert trong các luồng song song
        futures.append(executor.submit(batch_upsert, content_batch, title_batch))
    
    # Dùng as_completed để theo dõi tiến trình của các batch
    for future in as_completed(futures):
        try:
            result = future.result()  # Có thể kiểm tra lỗi nếu cần
            print("Một batch đã hoàn thành.")
        except Exception as e:
            print(f"Lỗi xảy ra: {e}")


Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.
Một batch đã hoàn thành.


In [11]:
collection_info = client.get_collection(collection_name="content_vectors")

print(f"Số lượng vectors trong collection 'words': {collection_info.points_count}")


Số lượng vectors trong collection 'words': 2400


In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from qdrant_client import QdrantClient, models
import numpy as np

# Kết nối tới Qdrant server
client = QdrantClient("http://localhost:6333")

# Hàm thêm dữ liệu vào Qdrant
def batch_upsert(points_batch, collection_name):
    points = [
        models.PointStruct(
            id=point["id"],
            vector=point["vector"],
            payload={"article_id": point["id"]}  # Tùy thuộc vào payload bạn muốn thêm
        )
        for point in points_batch
    ]
    client.upsert(collection_name=collection_name, points=points)

# Đọc file và chuẩn bị dữ liệu
def read_vectors_from_file(file_path):
    points = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            article_id = parts[0]
            vector = np.array(list(map(float, parts[1:])))
            points.append({"id": article_id, "vector": vector})
    return points

# Hàm chia dữ liệu thành các batch
def split_batches(data, batch_size):
    return [data[i:i + batch_size] for i in range(0, len(data), batch_size)]

# Đường dẫn đến file chứa ID và vector
file_path = 'contents_vectors.txt'

# Đọc toàn bộ dữ liệu từ file
all_points = read_vectors_from_file(file_path)

# Chia dữ liệu thành các batch (ví dụ: 100 vector mỗi batch)
batch_size = 100
batches = split_batches(all_points, batch_size)

# Sử dụng ThreadPoolExecutor để thêm dữ liệu song song
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for batch in batches:
        futures.append(executor.submit(batch_upsert, batch, "content_vectors"))  # Thêm vào collection contents
        # futures.append(executor.submit(batch_upsert, batch, "titles"))  # Thêm vào collection titles
    
    # Theo dõi tiến trình của các batch
    for future in as_completed(futures):
        try:
            result = future.result()
            print("Batch completed.")
        except Exception as e:
            print(f"Lỗi xảy ra: {e}")


Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch complete

In [5]:
import torch
from transformers import AutoTokenizer, AutoModel
from qdrant_client import QdrantClient, models
import numpy as np
from pyvi.ViTokenizer import tokenize

# Kết nối tới Qdrant server
client = QdrantClient("http://localhost:6333")

# Tải PhoBERT model và tokenizer
tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


# Hàm mã hóa văn bản thành vector sử dụng PhoBERT


Some weights of the model checkpoint at VoVanPhuc/sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.bias', 'mlp.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def encode_text(text, max_length=128):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]  # Vector từ token [CLS]
    return embeddings.squeeze().cpu().numpy()

# Hàm tìm kiếm các văn bản tương tự
def find_similar_texts(input_text, collection_name="content_vectors", limit=10):
    # Mã hóa đoạn văn bản đầu vào thành vector
    input_vector = encode_text(tokenize(input_text))
    
    # Gửi truy vấn tới Qdrant để tìm các vector tương tự
    search_result = client.search(
        collection_name=collection_name,
        query_vector=input_vector,
        limit=limit,
        with_payload=True
    )
    
    # Trả về các bài viết tương tự
    similar_texts = []
    for result in search_result:
        article_id = result.payload.get("article_id", "Unknown")
        score = result.score
        similar_texts.append({"article_id": article_id, "score": score})
    
    return similar_texts

# Ví dụ về đoạn văn bản đầu vào
input_text = "công an hối lộ"

# Tìm kiếm 10 văn bản tương tự
similar_articles = find_similar_texts(input_text)

# In kết quả
for idx, article in enumerate(similar_articles):
    print(f"{idx + 1}. Article ID: {article['article_id']}, Similarity Score: {article['score']}")


1. Article ID: 0c7c40975bc054caca743b767f7eae3f, Similarity Score: 0.5885566
2. Article ID: a7dc0d28978535360cc6367ca93ec925, Similarity Score: 0.5326917
3. Article ID: 2f50b89af15c090664b6d8f44a58de5f, Similarity Score: 0.5222285
4. Article ID: ee3bed7a6eb038b6d313d5d3bbee410c, Similarity Score: 0.5187587
5. Article ID: 8b4649fc71cdccf59af89e1a75b17a15, Similarity Score: 0.49611023
6. Article ID: 57f55ca44016c031c886a67a62d56f43, Similarity Score: 0.49611023
7. Article ID: 495adcf627f969fca990cd02e1d42a78, Similarity Score: 0.48978072
8. Article ID: aefec3490a6f2fe2052507f45b8c362c, Similarity Score: 0.48585397
9. Article ID: 07674ac676362abda2e274e73dc374a7, Similarity Score: 0.48241717
10. Article ID: 571eb76c8ad9a45eef52cb7a89b642bc, Similarity Score: 0.4807526
